In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

SETTING = kraft.read_json("../project.json")

if "project_directory_path" in SETTING:

    SETTING.update(
        kraft.read_json(os.path.join(SETTING["project_directory_path"], "project.json"))
    )

    SETTING["output_directory_path"] = os.path.join(
        SETTING["project_directory_path"], "output"
    )

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

w.columns.name = h.index.name

h.columns.name = SETTING["sample_alias"]

In [ ]:
for w_or_h, dataframe, drop_axis in (("w", w, 0), ("h", h, 1)):

    if drop_axis == 0:

        apply_axis = 1

    else:

        apply_axis = 0

    dataframe.drop(
        kraft.select_series_indices(
            dataframe.apply(kraft.compute_vector_entropy, axis=apply_axis),
            ">",
            fraction=SETTING["gps_map_{}_element_fraction_to_drop".format(w_or_h)],
            layout={
                "title": {"text": "Dropping {} Element".format(w_or_h.title())},
                "yaxis": {"title": "Entropy"},
            },
        ),
        axis=drop_axis,
        inplace=True,
    )

In [ ]:
gps_map = kraft.GPSMap(
    w=w.T,
    h=h,
    w_n_pull=SETTING["gps_map_w_n_pull"],
    w_pull_power=SETTING["gps_map_w_pull_power"],
    h_n_pull=SETTING["gps_map_h_n_pull"],
    h_pull_power=SETTING["gps_map_h_pull_power"],
    plot=False,
)

In [ ]:
for element_type, w_or_h, elements in (
    ("feature", "w", gps_map.w_elements),
    ("sample", "h", gps_map.h_elements),
):

    if elements is None:

        continue

    element_marker_size = SETTING["gps_map_{}_element_marker_size".format(w_or_h)]

    gps_map_directory_path = PATH["{}|gps_map/".format(w_or_h)]

    gps_map.plot(
        w_or_h,
        element_marker_size=element_marker_size,
        html_file_path=os.path.join(gps_map_directory_path, "gps_map.html"),
    )

    cluster_x_element_tsv_file_pth = PATH["{}|cluster_x_element.tsv".format(w_or_h)]

    if not os.path.isfile(cluster_x_element_tsv_file_pth):

        continue

    element_cluster = pd.read_csv(
        cluster_x_element_tsv_file_pth, sep="\t", index_col=0
    ).apply(lambda cluster_01: cluster_01.idxmax())

    gps_map.set_element_label(
        w_or_h,
        element_cluster,
        bandwidth_factor=SETTING["gps_map_{}_bandwidth_factor".format(w_or_h)],
    )

    gps_map.plot(
        w_or_h,
        element_marker_size=element_marker_size,
        html_file_path=os.path.join(gps_map_directory_path, "gps_map.html"),
    )

In [ ]:
kraft.write_gps_map(gps_map, PATH["gps_map.pickle.gz"])

In [ ]:
for element_type, w_or_h, elements in (
    ("feature", "w", gps_map.w_elements),
    ("sample", "h", gps_map.h_elements),
):

    element_value = pd.Series(
        (
            int(element in SETTING["{}s_to_peek".format(element_type)])
            for element in elements
        ),
        index=elements,
    )

    if not element_value.any():

        continue

    gps_map.plot(
        w_or_h,
        element_value=element_value,
        element_value_data_type="binary",
        element_marker_size=SETTING["gps_map_{}_element_marker_size".format(w_or_h)],
        element_value_binary_annotation={"font": {"size": 16}},
        html_file_path=os.path.join(
            PATH["{}|gps_map/".format(w_or_h)], "gps_map.peek.html"
        ),
    )